<a href="https://colab.research.google.com/github/ichakrabarty/big-data-challenge/blob/main/Level1data1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:13 http://security.ubuntu.com/ubuntu bionic-security 

In [12]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-10-01 22:06:45--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.72MB/s    in 0.5s    

2021-10-01 22:06:46 (1.72 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [13]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [22]:
from pyspark import SparkFiles
url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz'
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get('amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz'), sep='\t', header=True, inferSchema=True)

In [23]:
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games| 

In [28]:
df.count()

145431

In [27]:
dropna_df = df.dropna()
dropna_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games| 

In [29]:
dropna_df.count()

145425

In [30]:
from pyspark.sql.functions import col

review_id_table = dropna_df.select(["review_id", "customer_id", "product_id", "product_parent","review_date"])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RSH1OZ87OYK92|   21269168|B013PURRZW|     603406193| 2015-08-31|
|R1WFOQ3N9BO65I|     133437|B00F4CEHNK|     341969535| 2015-08-31|
| R3YOOS71KM5M9|   45765011|B00DNHLFQA|     951665344| 2015-08-31|
|R3R14UATT3OUFU|     113118|B004RMK5QG|     395682204| 2015-08-31|
| RV2W9SGDNQA2C|   22151364|B00G9BNLQE|     640460561| 2015-08-31|
|R3CFKLIZ0I2KOB|   22151364|B00IMIL498|     621922192| 2015-08-31|
|R1LRYU1V0T3O38|   38426028|B00S00IJH8|     215163395| 2015-08-31|
| R44QKV6FE5CJ2|    6057518|B004RMK4BC|     384246568| 2015-08-31|
|R2TX1KLPXXXNYS|   20715661|B00K59HKIQ|     384246568| 2015-08-31|
|R1JEEW4C6R89BA|   26540306|B00K59HKIQ|     384246568| 2015-08-31|
|R3B3UHK1FO0ERS|    8926809|B004774IPU|     151985175| 2015-08-31|
|R2GVSDHW513SS1|   31525534|B002LIT9EC|     695277014| 2015-08

In [31]:
products = dropna_df.select(["product_id", "product_title"])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B013PURRZW|Madden NFL 16 - X...|
|B00F4CEHNK| Xbox Live Gift Card|
|B00DNHLFQA|Command & Conquer...|
|B004RMK5QG|Playstation Plus ...|
|B00G9BNLQE|Saints Row IV - E...|
|B00IMIL498|Double Dragon: Ne...|
|B00S00IJH8|              Sims 4|
|B004RMK4BC|Playstation Netwo...|
|B00K59HKIQ|Playstation Netwo...|
|B00K59HKIQ|Playstation Netwo...|
|B004774IPU|Sid Meier's Civil...|
|B002LIT9EC|Build-a-lot 4: Po...|
|B008ALUBYQ|       Borderlands 2|
|B010KYDNDG|Minecraft for PC/...|
|B008ALUBYQ|       Borderlands 2|
|B004RMK4P8|Playstation Netwo...|
|B004RMK5QG|Playstation Plus ...|
|B00BI16Z22|Brink of Consciou...|
|B004VSTQ2A|Xbox Live Subscri...|
|B00GAC1D2G|Playstation Netwo...|
+----------+--------------------+
only showing top 20 rows



In [39]:
customers = dropna_df.groupby("customer_id").agg({"customer_id":"count"}).orderBy("count(customer_id)").withColumnRenamed("count(customer_id)","customer_count").show(truncate=False)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|25494099   |1             |
|12800594   |1             |
|11365485   |1             |
|45115558   |1             |
|14988646   |1             |
|26872164   |1             |
|38173350   |1             |
|41369838   |1             |
|46625306   |1             |
|13669205   |1             |
|14231016   |1             |
|2128552    |1             |
|712525     |1             |
|32831495   |1             |
|8211293    |1             |
|681559     |1             |
|9037272    |1             |
|50712554   |1             |
|49514578   |1             |
|936322     |1             |
+-----------+--------------+
only showing top 20 rows



In [41]:
vine_table = dropna_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RSH1OZ87OYK92|          2|            2|          3|   N|
|R1WFOQ3N9BO65I|          5|            0|          0|   N|
| R3YOOS71KM5M9|          5|            0|          0|   N|
|R3R14UATT3OUFU|          5|            0|          0|   N|
| RV2W9SGDNQA2C|          5|            0|          0|   N|
|R3CFKLIZ0I2KOB|          5|            0|          0|   N|
|R1LRYU1V0T3O38|          4|            0|          0|   N|
| R44QKV6FE5CJ2|          5|            0|          0|   N|
|R2TX1KLPXXXNYS|          5|            0|          0|   N|
|R1JEEW4C6R89BA|          5|            0|          0|   N|
|R3B3UHK1FO0ERS|          1|            0|          0|   N|
|R2GVSDHW513SS1|          5|            0|          0|   N|
|R1R1NT516PYT73|          5|            0|          0|   N|
|R3K624QDQKENN9|          4|            

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://<connection string>:5432/amazon_db"
config = {"user":"postgres", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}


In [ ]:
clean_user_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
clean_user_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
clean_user_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
clean_user_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)